<p style="font-family:Arial, serif;font-size:36px;font-style:normal;font-weight:bold;color:#0558ff;background-color:#ffffff;">Network traffic classification using CNN</p>

* [Loading and preparation of data](#section-one)
* [Creating model](#section-two)
* [Training model](#section-three)
 * [Loss and Accuracy plot](#section-three-one)
* [Testing and evaluating model](#section-four)
 * [Classification report](#section-four-one)
 * [Confusion matrix](#section-four-two)

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
np.random.seed(210)
import tensorflow as tf
import keras_tuner as kt
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Activation
from keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow import keras
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt
from laplotter import LossAccPlotter
from tensorflow.keras.layers import BatchNormalization
import pickle as pk
import prettytable
from prettytable import PrettyTable
import multiprocessing as mp
from utils import dict_name2label

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [ ]:
# preparation of necessary folders
folders = ['./models', './loss_acc_plots', './confusion_matrix']

for folder in folders:
    try:
        os.makedirs(folder)    
        print("Directory " , folder ,  " Created ")
    except FileExistsError:
        print("Directory " , folder ,  " already exists")   

#directory = './data' #for jupyter and colab notebooks
directory = '../input/pickles' #for kaggle notebook

In [ ]:
lock = mp.Lock()
counter = mp.Value('i', 0)

# Loading and preparation of data
<a id="section-one"></a>

In [ ]:
def gen_todo_list(directory):
    files = os.listdir(directory)
    todo_list = []
    for f in files:
      # Using only files listed in dict_name2label
      if f.split(".pickle")[0] in dict_name2label.keys():
        fullpath = os.path.join(directory, f)
        if os.path.isfile(fullpath):
          todo_list.append(fullpath)
    return todo_list

In [ ]:
def load(filename):
    with open(filename, 'rb') as f:
        data = pk.load(f)
    return data

In [ ]:
def load_data():
    max_data_nb = 10000
    todo_list = gen_todo_list(directory)
    ### ver 1 ###
    train_rate = 0.6
    val_rate = 0.2
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    X_test = []
    y_test = []

    for counter, filename in enumerate(todo_list):
        (tmpX, tmpy) = load(filename)      
        tmpX , tmpy = tmpX[:max_data_nb], tmpy[:max_data_nb]
        assert(len(tmpX) == len(tmpy))
        tmpX = processX(tmpX)
        train_num = int(len(tmpX) * train_rate)
        val_num = int(len(tmpX) * val_rate)
        X_train.extend(tmpX[:train_num])
        y_train.extend(tmpy[:train_num])
        X_val.extend(tmpX[train_num: train_num + val_num])
        y_val.extend(tmpy[train_num: train_num + val_num])
        X_test.extend(tmpX[train_num + val_num:])
        y_test.extend(tmpy[train_num + val_num:])
        print('\rLoading... {}/{}'.format(counter+1,len(todo_list)), end = '')
    print('\r{} Data loaded.               '.format(len(todo_list)))
    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
def processX(X):
    if True:
        X = np.array(X)
        lens = [len(x) for x in X] 
        maxlen = 1500
        tmpX = np.zeros((len(X), maxlen))
        mask = np.arange(maxlen) < np.array(lens)[:,None]
        tmpX[mask] = np.concatenate(X)
        return tmpX
    else:
        for i, x in enumerate(X):
            tmp_x = np.zeros((1500,))
            tmp_x[:len(x)] = x
            X[i] = tmp_x
        return X

In [ ]:
# load data
x_train, y_train, x_val, y_val, x_test, y_test = load_data()

# formatting arrays according to the dimensions of the required space, and formatting the type of variable (for numpy)
x_train = np.expand_dims(x_train, axis=2).astype(np.float32)
x_val = np.expand_dims(x_val, axis=2).astype(np.float32)
x_test = np.expand_dims(x_test, axis=2).astype(np.float32)

# one-hot-encoding application names
encoder = LabelEncoder()
encoder.fit(y_train)
class_labels = encoder.classes_

# number of classes in model
nb_classes = len(class_labels)

# number of data samples per training, validation and testing class
a = PrettyTable(["", "Application", "For training", "For validation", "For testing", "Total"])
# Alignment in the table
a.align[""] = "r"
a.align["Application"] = "l"
a.align["For training"] = "r"
a.align["For validation"] = "r"
a.align["For testing"] = "r"
a.align["Total"] = "r"
a.padding_width = 1

n = 0
zt_uk =0
zv_uk = 0
zts_uk = 0
labels = []
for ime in class_labels:
    labels.append(ime)
    zt = y_train.count(ime) # for training
    zv = y_val.count(ime) # for validation
    zts = y_test.count(ime) # for testing
    zt_uk = zt_uk + zt
    zv_uk = zv_uk + zv
    zts_uk = zts_uk + zts
    uk = zt + zv + zts
    a.add_row([n, ime, zt, zv, zts, uk]) 
    n += 1
a.add_row([' ', 'All total', zt_uk, zv_uk, zts_uk, zt_uk + zv_uk + zts_uk]) 
list_of_table_lines = a.get_string().split('\n')
horizontal_line = list_of_table_lines[0]
result_lines = 1
print("\n".join(list_of_table_lines[:-(result_lines + 1)]))
print(horizontal_line)
print("\n".join(list_of_table_lines[-(result_lines + 1):]))

encoded_y_train = encoder.transform(y_train)
y_train = np_utils.to_categorical(encoded_y_train)

encoded_y_test = encoder.transform(y_test)
y_test = np_utils.to_categorical(encoded_y_test)

encoded_y_val = encoder.transform(y_val)
y_val = np_utils.to_categorical(encoded_y_val)

# Creating model
<a id="section-two"></a>

In [ ]:
input_size = 1500
dropout = 0.2

# Build a model
model = Sequential()
model.add(Conv1D(60, 5, input_shape = (input_size,1), activation = 'relu'))
model.add(Dropout(dropout))
model.add(MaxPooling1D(2))
model.add(Flatten())
denses = [200, 100, 50]
for dense in denses:
    model.add(Dense(dense, activation = 'relu'))
    model.add(Dropout(dropout))
model.add(Dense(nb_classes, activation = 'softmax'))
model.summary()

## Training model
<a id="section-three"></a>

In [ ]:
print("Training CNN model:")

batch_size = 32
nb_epochs = 50

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly='true', optimizer=opt)

# location of the saved model
saved_model_file = 'models/cnn_model.h5'.format('conv1d-cnn')

# Keeping model in control points where function loss improves
checkpoint = ModelCheckpoint(saved_model_file, monitor='val_loss', save_best_only=True, verbose=1)
fit_history = model.fit(x_train, y_train, epochs=nb_epochs, batch_size=batch_size, validation_data=(x_val,y_val),  
                    callbacks=[checkpoint])

print("Training of CNN model is over.\n")

## Loss and Accuracy plot
<a id="section-three-one"></a>

In [ ]:
plotter = LossAccPlotter(title = 'Performanses of loss and accuracy for CNN model',
                         save_to_filepath='loss_acc_plots/cnn.png',
                         show_regressions=True,
                         show_averages=False,
                         show_loss_plot=True,
                         show_acc_plot=True,
                         show_plot_window=False,
                         x_label="Epoch")

num_epochs = len(fit_history.history['accuracy'])

for epoch in range(nb_epochs):
    acc_train = fit_history.history['accuracy'][epoch]
    loss_train =fit_history.history['loss'][epoch]
    acc_val = fit_history.history['val_accuracy'][epoch]
    loss_val = fit_history.history['val_loss'][epoch]

    plotter.add_values(epoch, loss_train=loss_train, acc_train=acc_train, loss_val=loss_val, acc_val=acc_val, redraw=False)

plotter.redraw()
plotter.block()

# Testing and evaluating model
<a id="section-four"></a>

## Classification report
<a id="section-four-one"></a>

In [ ]:
#model = load_model('models/cnn_model.h5')
model = load_model('./models/cnn_model.h5')
#loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Performance report for CNN model:")

preds = model.predict(x_test, batch_size=32,  verbose=0)

y_true_labels = [np.argmax(t) for t in y_test]
y_preds_labels = [np.argmax(t) for t in preds]

class_metric_report = classification_report(y_true_labels, y_preds_labels, target_names=class_labels, digits=4)
print(class_metric_report)

## Confusion matrix
<a id="section-four-two"></a>

In [ ]:
def plot_confusion_matrix(y_labels, preds, class_labels):

    y_true_labels = [np.argmax(t) for t in y_labels]
    y_preds_labels = [np.argmax(t) for t in preds]

    cm = confusion_matrix(y_true_labels, y_preds_labels, normalize='true')

    df_cm = pd.DataFrame(cm)
    plt.figure(figsize=(30,25))
    plt.xlabel('Predicted')
    plt.ylabel('Real')
    fig = sn.heatmap(df_cm, cmap='coolwarm', xticklabels=class_labels, 
        yticklabels=class_labels[0], linewidths=.5, annot=True, fmt=".2f")
    plt.show()
    pdf_filename = 'confusion_matrix/cnn-' + 'confusion_matrix.pdf'
    fig.get_figure().savefig(pdf_filename, dpi=400)
    return pdf_filename

In [ ]:
plot_confusion_matrix(y_test, preds, class_labels)